In [4]:
from pyspark.sql import *

from pyspark import SparkConf
from settings import warehouse_location
conf = SparkConf()
catalog_loc = warehouse_location
print(catalog_loc)

conf.set(
    "spark.jars.packages",
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.2")
conf.set("spark.sql.execution.pyarrow.enabled", "true")


file:///home/venumyneni/personal/projects/learn_pyspark/spark_warehouse/iceberg


In [5]:
catalog = "dev_catalog1"
schema = "nyc"
table = "taxis"

In [6]:
conf.set(f"spark.sql.catalog.{catalog}", "org.apache.iceberg.spark.SparkCatalog")
conf.set(f"spark.sql.catalog.{catalog}.type","hadoop")


conf.set(f"spark.sql.catalog.{catalog}.warehouse",catalog_loc )
conf.set("spark.sql.defaultCatalog",catalog)

spark = SparkSession.builder.config(conf=conf).getOrCreate()

24/01/06 14:36:22 WARN Utils: Your hostname, SG-LN-N1170 resolves to a loopback address: 127.0.1.1; using 192.168.29.196 instead (on interface wlp0s20f3)
24/01/06 14:36:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/venumyneni/.ivy2/cache
The jars for the packages stored in: /home/venumyneni/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b1909e19-fe81-4440-aa86-3b6edc588a1b;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.2 in central
:: resolution report :: resolve 89ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.4.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.ap

In [7]:
tab = spark.table(f"{catalog}.{schema}.{table}")

In [9]:
tab.show()

+---------+-------+-------------+-----------+------------------+
|vendor_id|trip_id|trip_distance|fare_amount|store_and_fwd_flag|
+---------+-------+-------------+-----------+------------------+
|        1|      2|         78.2|       NULL|              NULL|
+---------+-------+-------------+-----------+------------------+



In [10]:
type(tab)

pyspark.sql.dataframe.DataFrame

In [20]:
spark.sql(f"DESCRIBE TABLE {schema}.{table};")

DataFrame[col_name: string, data_type: string, comment: string]

In [25]:
spark.sql(f"SELECT * FROM {catalog}.{schema}.{table}.files;").show()


+-------+--------------------+-----------+-------+------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+
|content|           file_path|file_format|spec_id|record_count|file_size_in_bytes|        column_sizes|        value_counts|   null_value_counts|nan_value_counts|        lower_bounds|        upper_bounds|key_metadata|split_offsets|equality_ids|sort_order_id|    readable_metrics|
+-------+--------------------+-----------+-------+------------+------------------+--------------------+--------------------+--------------------+----------------+--------------------+--------------------+------------+-------------+------------+-------------+--------------------+
|      0|file:/home/venumy...|    PARQUET|      0|           1|              1411|{1 -> 46, 2 -> 46...|{1 -> 1, 2 -> 1, ...|{1 -> 0, 2 -> 0, ...|{4 -> 0, 3 -> 0

In [30]:
tab = spark.read.format("iceberg").load(f"/home/venumyneni/personal/projects/learn_pyspark/spark_warehouse/iceberg/nyc/taxis")

In [31]:
tab.show()

+---------+-------+-------------+-----------+------------------+
|vendor_id|trip_id|trip_distance|fare_amount|store_and_fwd_flag|
+---------+-------+-------------+-----------+------------------+
|        1|      2|         78.2|       NULL|              NULL|
+---------+-------+-------------+-----------+------------------+

